In [7]:
import pandas as pd

file_path = '../../real_data/trace/W2K8.TPCC.10-19-2007.12-51-PM.trace-filtered-target.txt'

In [8]:
#  read the file
trace = pd.read_csv(file_path, sep=' ')
trace.columns = ['ByteOffset', 'IOSize']
print(len(trace))
print(trace.head())

5899528
   ByteOffset  IOSize
0  3201773568    8192
1  3155169280    4096
2  3155156992    4096
3  3161305088    8192
4  3154132992    4096


In [9]:
# declare dataframe variable to store converted trace
converted_trace = pd.DataFrame(columns=['ByteOffset', 'IOSize'])
counter = 0
# READ each line of the trace
for index, row in trace.iterrows():
    converted_list = []
    # IOSize to sector number and save it to the new trace
    sector_number = row['IOSize'] // 512
    # add a sector in replace of io size
    converted_list.append(row['ByteOffset'])
    converted_list.append(sector_number)
    converted_trace.loc[counter] = converted_list
    counter += 1
    if counter % 100000 == 0:
        print(counter)
    if counter > 1600000:
        break
    
print(converted_trace.head())

    


100000
200000
300000
400000
500000
600000
700000
800000
900000
1000000
1100000
1200000
1300000
1400000
1500000
1600000
   ByteOffset  IOSize
0  3201773568      16
1  3155169280       8
2  3155156992       8
3  3161305088      16
4  3154132992       8


In [10]:
# Save the converted trace
converted_trace.to_csv('../../real_data/trace/W2K8.TPCC.10-19-2007.12-51-PM.trace-filtered-target-converted.txt', sep=' ', index=False)